In [1]:
import sys
sys.path.append('/vol/tensusers5/nhollain/s_clip_scripts')

import itertools
from main import main, format_checkpoint
from params import parse_args
import copy
import os
from tqdm import tqdm

def prep_str_args(str_args): # Code to parse the string style arguments, as shown below
    str_args = str_args.split('\n') # Split on newline
    str_args = [s.strip() for s in str_args] # Remove any whitespaces from the start and end of the strings
    # Split on the space between the parameter name and the value, e.g. '--name x' becomes ['--name', 'x']
    str_args = [s.split(' ') for s in str_args] 
    str_args = list(itertools.chain(*str_args)) # Flatten the resulting list of lists
    str_args = [s for s in str_args if len(s) > 0] # Remove arguments that are empty
    return str_args
    
def evaluate(checkpoint, epoch = None, kfold = -1):
    print('=> Resuming checkpoint {} (epoch {})'.format(checkpoint, epoch))
    checkpoint = checkpoint_path 
    if 'Fashion' in checkpoint:
        zeroshot_datasets = ["Fashion200k-SUBCLS", "Fashion200k-CLS", "FashionGen-CLS", "FashionGen-SUBCLS", "Polyvore-CLS", ]
        retrieval_datasets = ["FashionGen", "Polyvore", "Fashion200k",]
    else:
        zeroshot_datasets = ["RSICD-CLS", "UCM-CLS"] # "WHU-RS19", "RSSCN7", "AID" -> NOT WORKING bc of different data-loading workings
        retrieval_datasets = ["RSICD", "UCM", "Sydney"]
    
    for dataset in zeroshot_datasets:
        str_args = ['--name', checkpoint, '--imagenet-val', dataset, '--resume-epoch', str(epoch), '--k-fold', str(kfold)]
        args = parse_args(str_args)
        main(args)
    
    for dataset in retrieval_datasets:
        str_args = ['--name', checkpoint, '--val-data', dataset, '--resume-epoch', str(epoch), '--k-fold', str(kfold)]
        args = parse_args(str_args)
        main(args)

[nltk_data] Downloading package punkt to /home/nhollain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nhollain/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from collections import Counter

results = []
if os.path.exists('./eval.txt'):   
    with open('./eval.txt', 'r') as f:
        results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t')[0] for r in results if '\t' in r]
model_names = results
# Remove the timestamp from the model names, as well as the specific fold - rest of the name contains params
model_names = ['-'.join(m.split('-')[2:]).split('-fold')[0] for m in model_names]
model_names = dict(Counter(model_names))
# Show which models are already present in eval.txt
model_names

{'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_5-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_10-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_15-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_20-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_25-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_30-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_None-PL_None-vit_False-epochs_35-lr_0.00005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50-method_ours-kw_classname-AL.iter_None-PL_ot.image-vit_False-epochs_5-lr_0.0005-bs_64': 180,
 'data_RS-ALL-ratio_0.1-model_RN50

# Grid Search

In [4]:
# Do a grid search on the parameters
# NOTE: for active learning, save-freq should be set to 1
base_str_args = ''' --train-data RS-ALL
--label-ratio 0.1
--val-data RS-ALL
--imagenet-val RSICD-CLS 
--keyword-path keywords/RS/class-name.txt
--zeroshot-frequency 5  
--method base
--save-freq 5
--active-learning
'''

# Dictionary of values to gridsearch for hyperparam tuning
gridsearch_dict = {
    '--epochs' : [5], #list(range(15,36,5)) if 'active-learning' in base_str_args else [35], #[10,15,20,25,30,35],
    '--lr' : [5e-5], # 5e-4, 5e-6
    '--batch-size' : [64],
    '--al-iter': [10, 20, 30] #list(range(1,6,2)),
}

# This number is very specifically chosen because we have 9 folds for the datasets!
num_repeats = 9 
num_evals = 20 # How many evaluations are done with evaluate(...)

gridsearch_values = list(gridsearch_dict.values())
gridsearch_keys = list(gridsearch_dict.keys())
configs = list(itertools.product(*gridsearch_values))
print('Number of configs:', len(configs))

Number of configs: 3


In [ ]:
%%time
for c, config in enumerate(configs): # Gridsearch
    str_args = copy.deepcopy(base_str_args)
    # Add the gridsearch parameters to the arguments
    for i, param in enumerate(config):
        param_name = gridsearch_keys[i]
        str_args += '\n{} {}'.format(param_name, param)
        
    str_args = prep_str_args(str_args)
    print(str_args)
    args = parse_args(str_args)
    checkpoint_hypothetical = format_checkpoint(args)
    # Remove the timestamp from the hypothetical checkpoint, so we can compare to the params of other checkpoints
    checkpoint_params = '-'.join(checkpoint_hypothetical.split('-')[2:]).split('-fold')[0]
    # print(checkpoint_params)
    # Check if we've already trained the exact same model, correct the number of training iterations we still need to do
    if checkpoint_params in model_names:
        # The number of times to repeat depends on how often the model's been evaluated already
        start_repeat = int(model_names[checkpoint_params]/num_evals)
    else: # If we've never trained + evaluated the model before, just use num_repeats
        start_repeat = 0
    print('Config number {}: {} repeats'.format(c, num_repeats-start_repeat))
    for i in range(start_repeat, num_repeats):
        print()
        print('repeat number', i) 
        args = parse_args(str_args)
        args.k_fold = i
        # We compute here for which epochs we need to evaluate (based on for which epochs we checkpoint)
        epoch_freq = args.save_freq
        epochs = list(range(epoch_freq,args.epochs+1,epoch_freq))
        print('Epochs to checkpoint', epochs)
        # print('Args k fold (outside):' , args.k_fold)
        checkpoint_path = main(args) # Calls the main.py function of S-CLIP
        for epoch in epochs:
            evaluate(checkpoint_path, epoch = epoch, kfold = i)

['--train-data', 'RS-ALL', '--label-ratio', '0.1', '--val-data', 'RS-ALL', '--imagenet-val', 'RSICD-CLS', '--keyword-path', 'keywords/RS/class-name.txt', '--zeroshot-frequency', '5', '--method', 'base', '--save-freq', '5', '--active-learning', '--epochs', '5', '--lr', '5e-05', '--batch-size', '64', '--al-iter', '10']
Running on GPU: cuda:2
Config number 0: 9 repeats

repeat number 0
Running on GPU: cuda:2
Epochs to checkpoint [5]
kfold: 0
Log path: ./checkpoint/2023_11_15-15_56_27-data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_10-PL_None-vit_False-epochs_5-lr_0.00005-bs_64-fold_0
RS-ALL (split: val)	Sub-dataset sizes: [1092, 234, 62] (sum = 1388)
RSICD-CLS (split: val)	CLS size: 1092	RS-ALL (split: train)	Sub-dataset sizes: [8736, 1866, 493] (sum = 11095)
Labeled size: 110
Active learning...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5568/5568 [00:37<00:00, 149.24it/s]


Active Learning iteration: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:41<00:00,  8.32s/it]


CLS size: 1092	Active learning...


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.59s/it]


CLS size: 1092	Active learning...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5568/5568 [00:31<00:00, 177.06it/s]


Active Learning iteration: 10


 20%|██████████████████████████████▍                                                                                                                         | 1/5 [00:11<00:47, 11.81s/it]

# Training

In [5]:
fashion = False

if fashion:
    str_args = '''--train-data Fashion-ALL
            --label-ratio 0.1
            --val-data Fashion-ALL
            --keyword-path keywords/fashion/class-name.txt
            --epochs 10
            --method base  
    '''
else:
    str_args = ''' --train-data RS-ALL
            --label-ratio 0.1
            --val-data RS-ALL
            --imagenet-val RSICD-CLS \
            --keyword-path keywords/RS/class-name.txt
            --epochs 5
            --lr 5e-5
            --zeroshot-frequency 5  
            --method base
            --active-learning
            --al-iter 3
    '''
           # --active-learning
        # --al-iter 3

# Convert string arguments to a format that can be parsed by parse_args             
str_args = prep_str_args(str_args)
args = parse_args(str_args)

Running on GPU: cuda:0


In [ ]:
%%time
checkpoint_path = main(args)

Log path: ./checkpoint/2023_11_14-16_51_10-data_RS-ALL-ratio_0.1-model_RN50-method_base-kw_none-AL.iter_3-PL_None-vit_False-epochs_5-lr_0.00005-bs_64
RS-ALL (split: val)	Sub-dataset sizes: [1094, 210, 58] (sum = 1362)
RSICD-CLS (split: val)	CLS size: 1094	RS-ALL (split: train)	Sub-dataset sizes: [8734, 1680, 497] (sum = 10911)
Labeled size: 363
Active learning...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5504/5504 [00:37<00:00, 146.30it/s]


Active Learning iteration: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:51<00:00, 10.34s/it]


CLS size: 1094	Active learning...


 50%|████████████████████████████████████████████████████████████████████████▌                                                                        | 2752/5504 [00:15<00:15, 181.32it/s]

# Eval

In [ ]:
%%time
# evaluate(checkpoint_path)